In [4]:
import requests
from bs4 import BeautifulSoup

In [6]:
import torch
from sklearn.preprocessing import LabelEncoder

# 간단한 데이터 예시
question_encoder = LabelEncoder()
answer_encoder = LabelEncoder()
question_encoder.fit(['오늘 날씨 어때?', '날씨 알려줘', '날씨는?', '내일 날씨는?', '오늘 비 오나?'])
answer_encoder.fit(['오늘의 날씨는 맑습니다.', '오늘의 날씨는 흐립니다.', '오늘의 날씨는 비가 옵니다.', '내일은 구름 많습니다.', '오늘 비가 옵니다.'])

def encode_sentence(sentence, encoder):
    tokens = sentence.split()  # 간단한 토큰화
    encoded_tokens = [encoder.transform([token])[0] for token in tokens if token in encoder.classes_]
    return torch.tensor(encoded_tokens, dtype=torch.long).unsqueeze(1)  # (seq_len, batch_size)

def decode_sentence(encoded_sentence, decoder):
    tokens = encoded_sentence.squeeze().tolist()
    return ' '.join(decoder.inverse_transform(tokens))


In [7]:
import torch
import torch.nn as nn

class TransformerChatBot(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads, num_layers, hidden_dim):
        super(TransformerChatBot, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.transformer = nn.Transformer(
            d_model=embed_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers
        )
        self.fc_out = nn.Linear(embed_size, vocab_size)

    def forward(self, src, tgt):
        src_emb = self.embedding(src)
        tgt_emb = self.embedding(tgt)
        output = self.transformer(src_emb, tgt_emb)
        return self.fc_out(output)

# 모델 초기화
vocab_size = len(question_encoder.classes_)  # 어휘 크기
embed_size = 128
num_heads = 8
num_layers = 2
hidden_dim = 256

model = TransformerChatBot(vocab_size, embed_size, num_heads, num_layers, hidden_dim)
model.eval()  # 모델을 평가 모드로 전환


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


TransformerChatBot(
  (embedding): Embedding(5, 128)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
          )
          (linear1): Linear(in_features=128, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=128, bias=True)
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerDecoderLayer(
   

In [ ]:
import torch

# 대화 기록 초기화
messages = []

def generate_response(user_input):
    # 사용자 입력을 인코딩
    user_input_encoded = encode_sentence(user_input, question_encoder)

    # Transformer 모델을 사용하여 응답 생성 (임시로 랜덤 응답)
    # 실제로는 모델을 학습시켜야 합니다.
    response_encoded = torch.randint(0, vocab_size, (user_input_encoded.size(0), 1))  # 임시 응답
    reply = decode_sentence(response_encoded, answer_encoder)

    return reply

while True:
    message = input("User: ")
    if message:
        messages.append({"role": "user", "content": message})

        # 모델을 사용하여 응답 생성
        reply = generate_response(message)

        # 응답 출력 및 기록
        print(f"AIUS: {reply}")
        messages.append({"role": "assistant", "content": reply})


User: 날씨 알려줘
AIUS: 
